In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv('/kaggle/input/credit-card-customers/BankChurners.csv')
data

Visualize the relationship between Gender and Card_Category.

In [ ]:
gender=data['Gender'].to_numpy()
gender_index=[]
for v in np.unique(gender):
    gender_index.append(np.where(gender==v)[0])
gender_cat=np.unique(gender)
cm=['r','b']
bar_width = 0.2
for i,g in enumerate(gender_index):
    card=data['Card_Category'].to_numpy()[g]
    client_card_num=[]
    for j,c in enumerate(np.unique(card)):
        plt.text(j-.3 / 2+i*.3 ,len(np.where(card==c)[0]),str(len(np.where(card==c)[0])))
        client_card_num.append(len(np.where(card==c)[0]))
    index=np.arange(np.unique(card).shape[0])
    plt.xticks(index+.3 / 2 ,np.unique(card))
    plt.bar(index+i*0.2,client_card_num,bar_width,color=cm[i],label=gender_cat[i])
    
plt.legend() 
plt.grid(True)
plt.show()

Build model to predict clients' classifiction: Attrited or Existing.

In [ ]:
data=data.drop(columns=['CLIENTNUM'])
data

In [ ]:
data=pd.get_dummies(data,prefix=['Attrition_Flag','Gender','Education_Level','Marital_Status','Income_Category','Card_Category'])
data

In [ ]:
data.isnull().values.any()

In [ ]:
for i,c in enumerate(data.columns):
    print(i,c)

In [ ]:
data_arr=data.to_numpy()
y_origin=data_arr[:,16:18]
X_origin=np.delete(data_arr,[16,17],1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_origin, y_origin, test_size=0.20, random_state=12)

In [ ]:
import tensorflow as tf
model=tf.keras.Sequential([
        tf.keras.layers.Dense(1024, activation='elu',input_dim=X_train.shape[1],kernel_initializer='normal'),
        tf.keras.layers.Dense(256, activation='elu',kernel_initializer='normal'),
        tf.keras.layers.Dense(64, activation='elu',kernel_initializer='normal'),
        tf.keras.layers.Dense(16, activation='elu',kernel_initializer='normal'),
        tf.keras.layers.Dense(units = 2, activation='softmax')
    ])
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train,y_train,epochs=10,batch_size=60)

In [ ]:
y_pred=model.predict(X_test)
Y_pred=np.argmax(y_pred,axis=1)
Y_test=np.argmax(y_test,axis=1)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
ac=accuracy_score(Y_pred,Y_test)
cm=confusion_matrix(Y_pred,Y_test)

In [ ]:
ac